In [22]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.models import model_from_json

from skimage.transform import resize

In [23]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    ds = pydicom.dcmread(filename)  
    if ds.Modality=='DX' and (ds.PatientPosition in ['AP','PA']) and ds.BodyPartExamined=='CHEST':
        img = ds.pixel_array
        print('Load file {} ...'.format(filename))
        return img
    else:
        print('Image format is not compatible with the algorithm.')

    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std): 
    img=img/255.0  
    nor_img=(img-img_mean)/img_std 
    resized_img=resize(nor_img,(224,224))
    proc_img = np.zeros((224,224,3))

    proc_img[:, :, 0] = resized_img
    proc_img[:, :, 1] = resized_img
    proc_img[:, :, 2] = resized_img
    proc_img = np.resize(proc_img,(1,224,224,3))
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # todo
    # load json and create model
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(weight_path)
    print("Loaded model from disk")
    
    return loaded_model


# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # todo    
    prediction_pro = model.predict(img)
    if prediction_pro>=thresh:
        prediction=1
    else:
        prediction=0
    return prediction 

In [24]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'
weight_path = 'xray_class_my_model.best.hdf5'

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0.490
img_std = 0.248

my_model = load_model(model_path, weight_path)
thresh = 0.355

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Loaded model from disk
Load file test1.dcm ...
0
Load file test2.dcm ...
0
Load file test3.dcm ...
1
Image format is not compatible with the algorithm.
Image format is not compatible with the algorithm.
Image format is not compatible with the algorithm.


In [28]:
my_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Model)              (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 25089     
Total params: 14,739,777
Trainable params: 2,384,897
Non-trainable params: 12,354,880
_________________________________________________________________
